In [51]:
#mount Google drive on Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
#import the libraries needed for the project in alphabetical order
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')
print('Sucessfull')

Sucessfull


In [53]:
#data loader class
class LoadData:
  def __init__(self, dir_name, file_name):
    self.dir_name = dir_name
    self.file_name = file_name
    
 
  def read_csv(self):
    os.chdir(self.dir_name)
    df = pd.read_csv(self.file_name)
    return df

#an object of the Data
load_data = LoadData('/content/drive/My Drive/Colab Notebooks/Datasets', 'energydata_complete.csv')

In [54]:
#Read the comma-separated values (csv) file into DataFrame.
energydata_df = load_data.read_csv()
energydata_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [55]:
#check for missing values in the dataset
energydata_df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [56]:
#check for duplicate entries (rows) in the dataset
energydata_df.duplicated().sum()

0

In [57]:
#check if the labels are in the correct data types
energydata_df.dtypes

date            object
Appliances       int64
lights           int64
T1             float64
RH_1           float64
T2             float64
RH_2           float64
T3             float64
RH_3           float64
T4             float64
RH_4           float64
T5             float64
RH_5           float64
T6             float64
RH_6           float64
T7             float64
RH_7           float64
T8             float64
RH_8           float64
T9             float64
RH_9           float64
T_out          float64
Press_mm_hg    float64
RH_out         float64
Windspeed      float64
Visibility     float64
Tdewpoint      float64
rv1            float64
rv2            float64
dtype: object

>> The **_Date_** label is a string object instead of a datetime data type. There is no need to convert the datatype from string to datetime since this column will not be needed to build the Linear Regression model 

**_Question 12_**

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [58]:
#assign X and y values to the column label T2 and T6 respectively as specified in the question
X, y = energydata_df.T2, energydata_df.T6

In [59]:
#reshape the data using numpy
X = np.array(X).reshape(-1, 1)
y = np.array(y).reshape(-1, 1)

In [60]:
#instantiate an object of the LinearRegression and fit the model
linear_model = LinearRegression()
linear_model.fit(X, y)

LinearRegression()

In [61]:
#compute the predicted values and find the R^2 value
predicted_values = linear_model.predict(X)
r2_sc = r2_score(y, predicted_values)
print(f'The R^2 value in two decimal place is : {(r2_sc):.2f}')

The R^2 value in two decimal place is : 0.64


**_Question 13_** 

Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [62]:
#drop the following columns: ['date', 'lights']
energy_df = energydata_df.drop(columns = ['date', 'lights'])
energy_df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [63]:
#Normalize the dataset using the MinMaxScaler 
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_df), columns = energy_df.columns)
normalised_df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [64]:
#select the independent variable from the dataset (all columns except the target variable)
features_df = normalised_df.drop('Appliances', axis = 1)
features_df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [65]:
#set the target variable to the column: ['Appliances']
target_df = normalised_df[['Appliances']]

In [66]:
#split the data into train and test set (70-30)
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size = 0.3, random_state = 42)

In [67]:
#Fit the model, find the predicted values and calculate the mean absolute error
linear_model.fit(X_train, y_train)
predicted_values = linear_model.predict(X_test)
mae = mean_absolute_error(y_test, predicted_values)
print(f'The Mean Absolute Error (MAE) is: {(mae):.2f}')

The Mean Absolute Error (MAE) is: 0.05


**_Question 14_**

What is the Residual Sum of Squares (in two decimal places)?

In [68]:
#calculate the Residual Sum of Squares
rss = np.sum(np.square((y_test - predicted_values)))[0]
print(f'The Residual Sum of Squares (RSS) is: {(rss):.2f}')

The Residual Sum of Squares (RSS) is: 45.35


**_Question 15_**

What is the Root Mean Squared Error (in three decimal places)?

In [69]:
#Calculate the Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print(f'The Root Mean Squared Error (RMSE) is: {(rmse):.3f}')

The Root Mean Squared Error (RMSE) is: 0.088


**_Question 16_**

What is the Coefficient of Determination (in two decimal places)?

In [72]:
#calculate the Coefficient of Determination R^2
coeff_det = r2_score(y_test, predicted_values)
print(f'The Coefficient of Determination (R^2) is: {(coeff_det):.2f}')

The Coefficient of Determination (R^2) is: 0.15


**_Question 18_**

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?


In [73]:
#Ridge Regresion Model using 0.4 alpha value
ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(X_train, y_train)
ridge_predict = ridge_reg.predict(X_test)

ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_predict))
print(f'The RMSE using Ridge Regression is: {(ridge_rmse):.3f}')

The RMSE using Ridge Regression is: 0.088


>>>Using Ridge Regression Model with alpha value of 0.4, there is no change to the root squared error (RMSE)

**_Question 20_**

What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [74]:
#calculate the RMSE with lasso regression
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(X_train, y_train)
lasso_predict = lasso_reg.predict(X_test)
lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_predict))
print(f'The RMSE with Lasso Regression is: {(lasso_rmse):.3f}')

The RMSE with Lasso Regression is: 0.094
